### 本文件已经完成历史使命了

In [ ]:
using RigidBodyDynamics

In [45]:
urdf = "../urdf/biped_No1.urdf"
mechanism = parse_urdf(urdf, floating=true)

Spanning tree:
Vertex: world (root)
  Vertex: torso-base, Edge: torso-base_to_world
    Vertex: left-block, Edge: left_j1
      Vertex: left-thigh, Edge: left_j2
        Vertex: left-shank, Edge: left_j3
    Vertex: right-block, Edge: right_j1
      Vertex: right-thigh, Edge: right_j2
        Vertex: right-shank, Edge: right_j3
No non-tree joints.

In [57]:
joints(mechanism)
joint_bw, left_j1, right_j1, left_j2, right_j2, left_j3, right_j3 = joints(mechanism)

7-element Array{Joint{Float64,JT} where JT<:JointType{Float64},1}:
 Joint "torso-base_to_world": Quaternion floating joint    
 Joint "left_j1": Revolute joint with axis [1.0, 0.0, 0.0] 
 Joint "right_j1": Revolute joint with axis [1.0, 0.0, 0.0]
 Joint "left_j2": Revolute joint with axis [0.0, 1.0, 0.0] 
 Joint "right_j2": Revolute joint with axis [0.0, 1.0, 0.0]
 Joint "left_j3": Revolute joint with axis [0.0, 1.0, 0.0] 
 Joint "right_j3": Revolute joint with axis [0.0, 1.0, 0.0]

In [47]:
state = MechanismState(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

In [58]:
velocity_range(state, left_j1)
velocity(state, joint_bw)

6-element view(::Array{Float64,1}, 1:6) with eltype Float64:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [54]:
# get joint handles, and build a null controller
function m_control!(torques::AbstractVector, t, state::MechanismState)
    torques[velocity_range(state, left_j1)] .= 1.0
    torques[velocity_range(state, left_j2)] .= 0
    torques[velocity_range(state, left_j3)] .= 0
    torques[velocity_range(state, right_j1)] .= 0
    torques[velocity_range(state, right_j2)] .= 0
    torques[velocity_range(state, right_j3)] .= 0
end

m_control! (generic function with 1 method)

In [55]:
# set intial condition
zero_velocity!(state)
set_configuration!(state, left_j1, 0.1)
set_configuration!(state, left_j2, 0.2)
set_configuration!(state, left_j3, 0.1)
set_configuration!(state, right_j1, 0.1)
set_configuration!(state, right_j2, 0.2)
set_configuration!(state, right_j3, 0.1)

In [5]:
# begin simulation
final_time = 10.
ts, qs, vs = simulate(state, final_time, m_control!; Δt = 1e-3);

In [ ]:
# drawing server
using MeshCatMechanisms

mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf));
open(mvis)

In [ ]:
# run animation
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);